# 1. Energy data

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'energy_data'
load ('./data/energy_data.mat');
[ndata, ~] = size(M);
X = M(:,[1:25,27:28]);
Y = M(:,26);
R = randperm(ndata);        
xt = X(R(1:1735),:);
yt = Y(R(1:1735));
R(1:1735) = [];
x = X(R,:);          
y = Y(R);
n = 18000; nt = 1735;   %Ms = 15

dataset = energy_data


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'Y' ;
opts.Ms = 30 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/energymodel', 'models','t_dGP_train')

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/energymodel30');
models = s.models;


In [5]:
criterions = {'BCM','PoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans BCM
BCM: SMSE 0.00113564, MSLL -4.2641, NLPD -1.4172
Processing energy_data kmeans PoE
PoE: SMSE 0.00126984, MSLL -4.1376, NLPD -1.2907


## 1.1 RBCM vs TERBCM

### 1.1.1 RBCM baseline

In [6]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans RBCM
RBCM: SMSE 0.00099527, MSLL -0.4389, NLPD 2.4079


### 1.1.2 Grid q

In [7]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:2.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing energy_data kmeans TERBCM
TERBCM: SMSE 0.00109508, MSLL 0.2457, NLPD 3.0926 - q   0.20
TERBCM: SMSE 0.00107941, MSLL -1.1059, NLPD 1.7410 - q   0.40
TERBCM: SMSE 0.00105769, MSLL -1.3802, NLPD 1.4666 - q   0.60
TERBCM: SMSE 0.00102941, MSLL -1.1507, NLPD 1.6962 - q   0.80
TERBCM: SMSE 0.00099527, MSLL -0.4389, NLPD 2.4079 - q   1.00
TERBCM: SMSE 0.00095724, MSLL 0.9389, NLPD 3.7858 - q   1.20
TERBCM: SMSE 0.00091808, MSLL 3.3892, NLPD 6.2361 - q   1.40
TERBCM: SMSE 0.00088039, MSLL 7.6787, NLPD 10.5255 - q   1.60
TERBCM: SMSE 0.00084591, MSLL 15.2239, NLPD 18.0708 - q   1.80
TERBCM: SMSE 0.00081539, MSLL 28.6564, NLPD 31.5033 - q   2.00


### 1.1.3 Find single q

In [8]:
criterion = 'TERBCM';

In [9]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=0.6);

Optimizing q
Msize =

 Columns 1 through 11:

    600    619    426   1016    528    358    580    958    195    718    703

 Columns 12 through 22:

    655    803    640    849    989    568    787    827    717    249    573

 Columns 23 through 30:

    263    480    710    488    453    442    584    222

Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 0.6000
grad_q_norm =  63.024
grad_q_reg = -55.781
Optimizing q: remain iter - 999, q - 0.6019
grad_q_norm =  34.314
grad_q_reg = -42.952
Optimizing q: remain iter - 998, q - 0.6034
grad_q_norm =  60.829
grad_q_reg = -42.299
Optimizing q: remain iter - 997, q - 0.6049
grad_q_norm =  73.350
grad_q_reg = -67.103
Optimizing q: remain iter - 996, q - 0.6072
grad_q_norm =  39.450
grad_q_reg = -51.977
Optimizing q: remain iter - 995, q - 0.6090
grad_q_norm =  83.578
grad_q_reg = -41.393
Optimizing q: remain iter - 994, q - 0.6104
grad_q_norm =  42.879
grad_q_reg = -53.675
Optimizing q: remain iter - 993

Optimizing q: remain iter - 910, q - 0.6888
grad_q_norm =  62.872
grad_q_reg = -24.222
Optimizing q: remain iter - 909, q - 0.6897
grad_q_norm =  0.87009
grad_q_reg = -11.220
Optimizing q: remain iter - 908, q - 0.6901
grad_q_norm =  10.081
grad_q_reg = -20.541
Optimizing q: remain iter - 907, q - 0.6908
grad_q_norm =  21.363
grad_q_reg = -22.337
Optimizing q: remain iter - 906, q - 0.6915
grad_q_norm =  13.618
grad_q_reg = -28.325
Optimizing q: remain iter - 905, q - 0.6925
grad_q_norm = -1.5435
grad_q_reg = -8.6882
Optimizing q: remain iter - 904, q - 0.6928
grad_q_norm =  0.51507
grad_q_reg = -12.128
Optimizing q: remain iter - 903, q - 0.6932
grad_q_norm =  65.202
grad_q_reg = -16.981
Optimizing q: remain iter - 902, q - 0.6938
grad_q_norm =  8.6095
grad_q_reg = -20.235
Optimizing q: remain iter - 901, q - 0.6945
grad_q_norm =  2.2785
grad_q_reg = -6.2798
Optimizing q: remain iter - 900, q - 0.6947
grad_q_norm = -10.019
grad_q_reg =  11.072
Optimizing q: remain iter - 899, q - 0.69

Optimizing q: remain iter - 816, q - 0.7305
grad_q_norm = -12.981
grad_q_reg =  4.8720
Optimizing q: remain iter - 815, q - 0.7303
grad_q_norm = -7.6290
grad_q_reg =  0.27553
Optimizing q: remain iter - 814, q - 0.7303
grad_q_norm = -22.256
grad_q_reg =  10.371
Optimizing q: remain iter - 813, q - 0.7300
grad_q_norm = -22.140
grad_q_reg =  13.139
Optimizing q: remain iter - 812, q - 0.7295
grad_q_norm = -33.451
grad_q_reg =  20.268
Optimizing q: remain iter - 811, q - 0.7288
grad_q_norm =  15.219
grad_q_reg = -23.859
Optimizing q: remain iter - 810, q - 0.7296
grad_q_norm = -1.6810
grad_q_reg = -10.853
Optimizing q: remain iter - 809, q - 0.7300
grad_q_norm =  2.5005
grad_q_reg = -12.696
Optimizing q: remain iter - 808, q - 0.7304
grad_q_norm =  20.045
grad_q_reg = -15.478
Optimizing q: remain iter - 807, q - 0.7310
grad_q_norm = -1.8936
grad_q_reg = -6.2284
Optimizing q: remain iter - 806, q - 0.7312
grad_q_norm =  9.2607
grad_q_reg = -15.217
Optimizing q: remain iter - 805, q - 0.731

Optimizing q: remain iter - 722, q - 0.7412
grad_q_norm = -27.946
grad_q_reg =  8.6598
Optimizing q: remain iter - 721, q - 0.7409
grad_q_norm =  6.8457
grad_q_reg = -12.014
Optimizing q: remain iter - 720, q - 0.7413
grad_q_norm =  14.915
grad_q_reg = -20.967
Optimizing q: remain iter - 719, q - 0.7420
grad_q_norm = -27.447
grad_q_reg =  12.161
Optimizing q: remain iter - 718, q - 0.7416
grad_q_norm = -2.0233
grad_q_reg = -3.3489
Optimizing q: remain iter - 717, q - 0.7417
grad_q_norm = -5.3683
grad_q_reg =  10.760
Optimizing q: remain iter - 716, q - 0.7413
grad_q_norm = -25.130
grad_q_reg =  14.858
Optimizing q: remain iter - 715, q - 0.7408
grad_q_norm = -36.713
grad_q_reg =  24.439
Optimizing q: remain iter - 714, q - 0.7400
grad_q_norm =  2.5548
grad_q_reg = -7.6402
Optimizing q: remain iter - 713, q - 0.7402
grad_q_norm =  48.760
grad_q_reg = -14.266
Optimizing q: remain iter - 712, q - 0.7407
grad_q_norm = -25.770
grad_q_reg =  17.895
Optimizing q: remain iter - 711, q - 0.7401

Optimizing q: remain iter - 628, q - 0.7487
grad_q_norm =  2.6956
grad_q_reg = -5.8979
Optimizing q: remain iter - 627, q - 0.7489
grad_q_norm =  21.357
grad_q_reg = -24.645


...


grad_q_reg =  11.265
Optimizing q: remain iter - 432, q - 0.7452
grad_q_norm =  20.648
grad_q_reg = -15.540
Optimizing q: remain iter - 431, q - 0.7457
grad_q_norm =  33.457
grad_q_reg = -17.049
Optimizing q: remain iter - 430, q - 0.7463
grad_q_norm =  2.3506
grad_q_reg = -3.3225
Optimizing q: remain iter - 429, q - 0.7464
grad_q_norm =  1.8803
grad_q_reg = -8.7040
Optimizing q: remain iter - 428, q - 0.7467
grad_q_norm =  8.6465
grad_q_reg = -5.6417
Optimizing q: remain iter - 427, q - 0.7469
grad_q_norm =  16.576
grad_q_reg = -12.944
Optimizing q: remain iter - 426, q - 0.7474
grad_q_norm = -2.3238
grad_q_reg = -2.9333
Optimizing q: remain iter - 425, q - 0.7475
grad_q_norm = -39.940
grad_q_reg =  23.148
Optimizing q: remain iter - 424, q - 0.7467
grad_q_norm =  9.6156
grad_q_reg = -19.524
Optimizing q: remain iter - 423, q - 0.7473
grad_q_norm =  24.853
grad_q_reg = -18.015
Optimizing q: remain iter - 422, q - 0.7480
grad_q_norm = -19.590
grad_q_reg =  9.5090
Optimizing q: remain 

grad_q_reg =  13.048
Optimizing q: remain iter - 338, q - 0.7493
grad_q_norm = -1.7283
grad_q_reg =  1.9158
Optimizing q: remain iter - 337, q - 0.7493
grad_q_norm =  5.8687
grad_q_reg = -4.2945
Optimizing q: remain iter - 336, q - 0.7494
grad_q_norm =  0.26577
grad_q_reg = -6.2852
Optimizing q: remain iter - 335, q - 0.7496
grad_q_norm = -40.374
grad_q_reg =  21.695
Optimizing q: remain iter - 334, q - 0.7489
grad_q_norm = -5.0991
grad_q_reg = -5.6140
Optimizing q: remain iter - 333, q - 0.7491
grad_q_norm =  5.4274
grad_q_reg = -6.3062
Optimizing q: remain iter - 332, q - 0.7493
grad_q_norm = -29.690
grad_q_reg =  20.150
Optimizing q: remain iter - 331, q - 0.7486
grad_q_norm = -26.870
grad_q_reg =  14.474
Optimizing q: remain iter - 330, q - 0.7481
grad_q_norm =  1.8289
grad_q_reg = -10.185
Optimizing q: remain iter - 329, q - 0.7484
grad_q_norm =  12.285
grad_q_reg = -20.911
Optimizing q: remain iter - 328, q - 0.7492
grad_q_norm = -5.2709
grad_q_reg = -8.1575
Optimizing q: remain 

grad_q_reg = -14.006
Optimizing q: remain iter - 244, q - 0.7522
grad_q_norm = -32.082
grad_q_reg =  10.981
Optimizing q: remain iter - 243, q - 0.7518
grad_q_norm = -7.8976
grad_q_reg = -3.7724
Optimizing q: remain iter - 242, q - 0.7519
grad_q_norm = -20.925
grad_q_reg =  15.821
Optimizing q: remain iter - 241, q - 0.7514
grad_q_norm =  10.729
grad_q_reg = -16.241
Optimizing q: remain iter - 240, q - 0.7519
grad_q_norm = -0.63773
grad_q_reg = -4.7855
Optimizing q: remain iter - 239, q - 0.7521
grad_q_norm = -21.616
grad_q_reg =  8.6460
Optimizing q: remain iter - 238, q - 0.7518
grad_q_norm = -24.628
grad_q_reg =  16.849
Optimizing q: remain iter - 237, q - 0.7512
grad_q_norm =  13.911
grad_q_reg = -13.318
Optimizing q: remain iter - 236, q - 0.7517
grad_q_norm =  41.762
grad_q_reg = -18.362
Optimizing q: remain iter - 235, q - 0.7523
grad_q_norm =  2.9573
grad_q_reg =  0.89458
Optimizing q: remain iter - 234, q - 0.7523
grad_q_norm =  20.493
grad_q_reg = -11.656
Optimizing q: remain

grad_q_reg = -15.922
Optimizing q: remain iter - 150, q - 0.7476
grad_q_norm =  5.5271
grad_q_reg =  0.60191
Optimizing q: remain iter - 149, q - 0.7476
grad_q_norm =  12.866
grad_q_reg = -18.890
Optimizing q: remain iter - 148, q - 0.7482
grad_q_norm =  17.798
grad_q_reg = -9.1776
Optimizing q: remain iter - 147, q - 0.7485
grad_q_norm =  6.2219
grad_q_reg = -13.322
Optimizing q: remain iter - 146, q - 0.7490
grad_q_norm =  11.067
grad_q_reg = -5.8669
Optimizing q: remain iter - 145, q - 0.7492
grad_q_norm = -22.780
grad_q_reg =  6.4470
Optimizing q: remain iter - 144, q - 0.7490
grad_q_norm =  18.018
grad_q_reg = -10.626
Optimizing q: remain iter - 143, q - 0.7493
grad_q_norm = -24.087
grad_q_reg =  18.864
Optimizing q: remain iter - 142, q - 0.7487
grad_q_norm = -29.176
grad_q_reg =  25.681
Optimizing q: remain iter - 141, q - 0.7478
grad_q_norm =  3.5820
grad_q_reg = -15.906
Optimizing q: remain iter - 140, q - 0.7483
grad_q_norm =  2.7464
grad_q_reg = -6.7008
Optimizing q: remain 

Optimizing q: remain iter - 56, q - 0.7504
grad_q_norm = -12.211
grad_q_reg =  3.9875
Optimizing q: remain iter - 55, q - 0.7503
grad_q_norm =  7.9904
grad_q_reg = -11.432
Optimizing q: remain iter - 54, q - 0.7507
grad_q_norm = -11.082
grad_q_reg =  1.3816
Optimizing q: remain iter - 53, q - 0.7506
grad_q_norm =  11.643
grad_q_reg = -8.2099
Optimizing q: remain iter - 52, q - 0.7509
grad_q_norm =  0.41785
grad_q_reg = -9.4941
Optimizing q: remain iter - 51, q - 0.7513
grad_q_norm = -0.53633
grad_q_reg = -7.0800
Optimizing q: remain iter - 50, q - 0.7515
grad_q_norm = -30.483
grad_q_reg =  21.962
Optimizing q: remain iter - 49, q - 0.7507
grad_q_norm = -20.374
grad_q_reg = -1.3127
Optimizing q: remain iter - 48, q - 0.7508
grad_q_norm =  24.968
grad_q_reg = -10.758
Optimizing q: remain iter - 47, q - 0.7512
grad_q_norm = -38.053
grad_q_reg =  28.372
Optimizing q: remain iter - 46, q - 0.7502
grad_q_norm = -23.119
grad_q_reg =  8.1633
Optimizing q: remain iter - 45, q - 0.7499
grad_q_no

In [10]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.1.4 Find multi q

In [11]:
criterion = 'TERBCM';

In [12]:
printf('Optimizing q\r\n');
iqs = q + 0.01 * rand(opts.Ms,1)
qs = mul_grad_q(models, criterion, opts, iter=1000, lr=1e-7, lambda=1e6, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
iqs =

   0.75667
   0.76006
   0.75115
   0.75681
   0.75891
   0.75969
   0.75839
   0.75116
   0.75288
   0.75658
   0.75602
   0.75449
   0.75114
   0.75182
   0.75291
   0.75326
   0.75060
   0.75501
   0.75126
   0.75393
   0.76028
   0.75500
   0.75510
   0.75951
   0.75930
   0.75718
   0.75788
   0.75516
   0.75382
   0.76028

Start preprocessing...
Finish preprocessing...
Optimizing qs: remain iter - 1000
qs =

   0.75667
   0.76006
   0.75115
   0.75681
   0.75891
   0.75969
   0.75839
   0.75116
   0.75288
   0.75658
   0.75602
   0.75449
   0.75114
   0.75182
   0.75291
   0.75326
   0.75060
   0.75501
   0.75126
   0.75393
   0.76028
   0.75500
   0.75510
   0.75951
   0.75930
   0.75718
   0.75788
   0.75516
   0.75382
   0.76028

qs_dist =  0.0055033
Optimizing qs: remain iter - 999
qs =

   0.75374
   0.76066
   0.75172
   0.75706
   0.75893
   0.76000
   0.75884
   0.75146
   0.75354
   0.75700
   0.75668
   0.75518
   0.75150
   0.74933
   0.75004
   0.7

   0.76469

qs_dist =  0.0064743
Optimizing qs: remain iter - 980
qs =

   0.71655
   0.75862
   0.76089
   0.75230
   0.76085
   0.75717
   0.75400
   0.74596
   0.75958
   0.75165
   0.76468
   0.75971
   0.74913
   0.74371
   0.74204
   0.75914
   0.73477
   0.74920
   0.74465
   0.76126
   0.76432
   0.76097
   0.74968
   0.76696
   0.75229
   0.76403
   0.76665
   0.75956
   0.76186
   0.76341

qs_dist =  0.0022057
Optimizing qs: remain iter - 979
qs =

   0.71545
   0.75862
   0.76094
   0.75300
   0.76140
   0.75725
   0.75405
   0.74664
   0.75995
   0.75169
   0.76505
   0.76008
   0.74915
   0.74433
   0.74213
   0.75914
   0.73506
   0.74981
   0.74477
   0.76184
   0.76457
   0.76099
   0.74974
   0.76700
   0.75234
   0.76445
   0.76691
   0.75956
   0.76194
   0.76408

qs_dist =  0.0043581
Optimizing qs: remain iter - 978
qs =

   0.71370
   0.75925
   0.76158
   0.75328
   0.76155
   0.75780
   0.75422
   0.74707
   0.76023
   0.75201
   0.76570
   0.76076
   0.74962
   

   0.75854
   0.76760
   0.76717

qs_dist =  0.0038888
Optimizing qs: remain iter - 959
qs =

   0.69016
   0.74927
   0.76996
   0.75268
   0.76507
   0.75781
   0.75573
   0.74429
   0.76616
   0.74849
   0.76276
   0.76358
   0.75044
   0.74864
   0.74649
   0.76386
   0.73519
   0.75030

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




   0.74982
   0.76756
   0.75854
   0.75174
   0.74851
   0.77218
   0.73461
   0.77307
   0.77032
   0.74682
   0.74818
   0.74242
   0.76883
   0.72983
   0.74707
   0.75080
   0.76429
   0.77227
   0.77266
   0.75414
   0.77689
   0.75109
   0.77832
   0.77749
   0.76187
   0.77172
   0.76740

qs_dist =  0.0037582
Optimizing qs: remain iter - 935
qs =

   0.66416
   0.74078
   0.78066
   0.75049
   0.76756
   0.75875
   0.75180
   0.74918
   0.77219
   0.73331
   0.77370
   0.77096
   0.74682
   0.74826
   0.74238
   0.76883
   0.72971
   0.74715
   0.75152
   0.76452
   0.77286
   0.77309
   0.75401
   0.77705
   0.75182
   0.77885
   0.77799
   0.76165
   0.77182
   0.76744

qs_dist =  0.0071006
Optimizing qs: remain iter - 934
qs =

   0.66260
   0.74153
   0.78130
   0.75049
   0.76783
   0.75875
   0.75192
   0.74637
   0.77213
   0.73367
   0.77428
   0.77150
   0.74698
   0.74691
   0.74021
   0.76946
   0.72758
   0.74437
   0.74910
   0.76442
   0.77317
   0.77371
   0.752

...

   0.84694
   0.69751
   0.78840
   0.77706
   0.74185
   0.75816
   0.72776
   0.88011
   0.69068
   0.75993
   0.72687
   0.85644
   0.76258
   0.81610
   0.74138
   0.80918
   0.75657
   0.79404
   0.88531
   0.84622
   0.86377
   0.80113

qs_dist =  0.0040462
Optimizing qs: remain iter - 376
qs =

   0.58900
   0.72855
   0.90761
   0.71804
   0.81884
   0.76276
   0.73808
   0.72098
   0.84717
   0.69612
   0.78832
   0.77679
   0.74076
   0.75868
   0.72718
   0.88018
   0.68914
   0.76054
   0.72733
   0.85694
   0.76116
   0.81610
   0.74082
   0.80757
   0.75701
   0.79385
   0.88548
   0.84613
   0.86409
   0.80155

qs_dist =  0.0029661
Optimizing qs: remain iter - 375
qs =

   0.58969
   0.72893
   0.90796
   0.71795
   0.81922
   0.76343
   0.73880
   0.72109
   0.84751
   0.69691
   0.78894
   0.77744
   0.74145
   0.75902
   0.72788
   0.88020
   0.68999
   0.76088
   0.72809
   0.85694
   0.76186
   0.81662
   0.74157
   0.80803
   0.75772
   0.79446
   0.88587
   0.8462

Optimizing qs: remain iter - 239
qs =

   0.58506
   0.73364
   0.92107
   0.70716
   0.83507
   0.75600
   0.72933
   0.71448
   0.86030
   0.69293
   0.80317
   0.78098
   0.73225
   0.75212
   0.72170
   0.89986
   0.68706
   0.76021
   0.71885
   0.86085
   0.77424
   0.81461
   0.72994
   0.81622
   0.75188
   0.80124
   0.90060
   0.86246
   0.87664
   0.80792

qs_dist =  0.0047790
Optimizing qs: remain iter - 238
qs =

   0.58472
   0.73392
   0.92137
   0.70716
   0.83562
   0.75669
   0.73004
   0.71220
   0.86024
   0.69366
   0.80375
   0.78159
   0.73288
   0.75120
   0.72248
   0.90028
   0.68770
   0.75723
   0.71926
   0.85991
   0.77491
   0.81494
   0.73072
   0.81654
   0.75257
   0.80175
   0.90078
   0.86249
   0.87709
   0.80744

qs_dist =  0.0044622
Optimizing qs: remain iter - 237
qs =

   0.58473
   0.73315
   0.92168
   0.70769
   0.83562
   0.75460
   0.72778
   0.71250
   0.86034
   0.69184
   0.80419
   0.78204
   0.73196
   0.75158
   0.72174
   0.90030
   

qs_dist =  0.0038438
Optimizing qs: remain iter - 218
qs =

   0.58379
   0.72864
   0.92322
   0.70914
   0.83786
   0.75059
   0.72259
   0.70957
   0.86126
   0.68873
   0.80756
   0.78123
   0.72982
   0.75202
   0.71702
   0.90289
   0.68775
   0.75830
   0.71938
   0.86209
   0.77078
   0.80908
   0.72436
   0.81042
   0.75131
   0.80529
   0.90473
   0.85847
   0.88039
   0.80949

qs_dist =  0.0048780
Optimizing qs: remain iter - 217
qs =

   0.58359
   0.72922
   0.92356
   0.70914
   0.83839
   0.75126
   0.72339
   0.70821
   0.86051
   0.68960
   0.80815
   0.78184
   0.73055
   0.75010
   0.71758
   0.90332
   0.68841
   0.75535
   0.71895
   0.86172
   0.77144
   0.80948
   0.72498
   0.81077
   0.75158
   0.80590
   0.90459
   0.85852
   0.88031
   0.80798

qs_dist =  0.0037575
Optimizing qs: remain iter - 216
qs =

   0.58410
   0.72995
   0.92394
   0.70932
   0.83875
   0.75192
   0.72406
   0.70868
   0.86055
   0.69047
   0.80874
   0.78248
   0.73131
   0.74903
   0

   0.88395
   0.80774

qs_dist =  0.0051604
Optimizing qs: remain iter - 197
qs =

   0.58220
   0.72973
   0.92605
   0.70996
   0.83632
   0.75129
   0.72208
   0.71032
   0.86020
   0.68820
   0.80702
   0.77185
   0.73218
   0.75135
   0.71998
   0.90154
   0.68564
   0.75654
   0.71756
   0.86618
   0.76563
   0.80782
   0.72360
   0.80810
   0.75574
   0.79623
   0.89623
   0.86237
   0.88423
   0.80811

qs_dist =  0.0059962
Optimizing qs: remain iter - 196
qs =

   0.58265
   0.72973
   0.92605
   0.71077
   0.83567
   0.75130
   0.72209
   0.71109
   0.86020
   0.68820
   0.80714
   0.77199
   0.73218
   0.75079
   0.72000
   0.89577
   0.68571
   0.75685
   0.71775
   0.86646
   0.76566
   0.80782
   0.72372
   0.80810
   0.75584
   0.79633
   0.89638
   0.86237
   0.88466
   0.80823

qs_dist =  0.0022274
Optimizing qs: remain iter - 195
qs =

   0.58345
   0.72974
   0.92614
   0.71160
   0.83604
   0.75147
   0.72227
   0.71192
   0.86045
   0.68837
   0.80768
   0.77259
   

   0.69472
   0.85973
   0.75053
   0.73411
   0.69071
   0.86448
   0.69227
   0.81959
   0.77581
   0.73887
   0.75254
   0.71713
   0.92477
   0.68335
   0.74292
   0.72094
   0.86804
   0.75809
   0.80613
   0.73948
   0.80601
   0.75631
   0.79575
   0.88976
   0.87564
   0.88955
   0.80304

qs_dist =  0.0048251


In [13]:
qs
save('./res/terbcmenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   0.58600
   0.74070
   0.94776
   0.69540
   0.85945
   0.75069
   0.73447
   0.69160
   0.86326
   0.69256
   0.82012
   0.77644
   0.73906
   0.74968
   0.71768
   0.92519
   0.68404
   0.74050
   0.72153
   0.86744
   0.75840
   0.80618
   0.74019
   0.80605
   0.75702
   0.79634
   0.89022
   0.87564
   0.88818
   0.80304

TERBCM: SMSE 0.00102249, MSLL -1.2663, NLPD 1.5806


## 1.2 GRBCM vs TEGRBCM

### 1.2.1 GRBCM baseline

In [14]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GRBCM
GRBCM: SMSE 0.00110323, MSLL -4.7444, NLPD -1.8975


### 1.2.2 Grid q

In [15]:
criterions = {'TEGRBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:2.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegrbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGRBCM
TEGRBCM: SMSE 0.00218277, MSLL -4.5226, NLPD -1.6758 - q   0.20
TEGRBCM: SMSE 0.00192482, MSLL -4.5824, NLPD -1.7355 - q   0.40
TEGRBCM: SMSE 0.00160054, MSLL -4.6669, NLPD -1.8200 - q   0.60
TEGRBCM: SMSE 0.00131082, MSLL -4.7407, NLPD -1.8938 - q   0.80
TEGRBCM: SMSE 0.00110323, MSLL -4.7444, NLPD -1.8975 - q   1.00
TEGRBCM: SMSE 0.00099626, MSLL -4.5613, NLPD -1.7144 - q   1.20
TEGRBCM: SMSE 0.00098432, MSLL -3.9637, NLPD -1.1169 - q   1.40
TEGRBCM: SMSE 0.00104077, MSLL -2.5095, NLPD 0.3374 - q   1.60
TEGRBCM: SMSE 0.00112809, MSLL 0.6676, NLPD 3.5145 - q   1.80
TEGRBCM: SMSE 0.00121376, MSLL 7.2876, NLPD 10.1344 - q   2.00


### 1.2.3 Find single q

In [29]:
criterion = 'TEGRBCM';

In [30]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=1.0);

Optimizing q
Msize =

 Columns 1 through 11:

    600    619    426   1016    528    358    580    958    195    718    703

 Columns 12 through 22:

    655    803    640    849    989    568    787    827    717    249    573

 Columns 23 through 30:

    263    480    710    488    453    442    584    222

Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 1.0000
    grad_q at line 111 column 16
grad_q_norm = NaN
grad_q_reg = NaN
Optimizing q: remain iter - 999, q - 0.2000
grad_q_norm =  0.051607
grad_q_reg =  0.011015
Optimizing q: remain iter - 998, q - 0.2000
grad_q_norm =  0.099567
grad_q_reg =  0.059904
Optimizing q: remain iter - 997, q - 0.2000
grad_q_norm = -0.10992
grad_q_reg =  0.041907
Optimizing q: remain iter - 996, q - 0.2000
grad_q_norm = -0.048261
grad_q_reg =  0.0021153
Optimizing q: remain iter - 995, q - 0.2000
grad_q_norm = -0.038319
grad_q_reg =  0.0089952
Optimizing q: remain iter - 994, q - 0.2000
grad_q_norm =  0.032016
grad

grad_q_norm = -0.66750
grad_q_reg =  0.029986
Optimizing q: remain iter - 921, q - 0.2000
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 920, q - 0.2000
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 919, q - 0.2000
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 918, q - 0.2000
Optimizing q: remain iter - 918, q - 0.2000
grad_q_norm = -0.034430
grad_q_reg =  0.029659
Optimizing q: remain iter - 917, q - 0.2000
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 916, q - 0.2000
grad_q_norm = -0.52684
grad_q_reg =  0.056038
Optimizing q: remain iter - 915, q - 0.2000
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 914, q - 0.2000
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 913, q - 0.2000
grad_q_norm = -0.20161
grad_q_reg =  0.034799
Optimizing q: remain iter - 912, q - 0.2000
grad_q_norm =    5.8878e-04
grad_q_reg =    1.3279e-05
Optimizing q: remain iter - 911, q - 0.2000
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: r

grad_q_norm = -0.026323
grad_q_reg =  0.011040
Optimizing q: remain iter - 831, q - 0.1999
grad_q_norm = -0.040080
grad_q_reg =  0.0098018
Optimizing q: remain iter - 830, q - 0.1999
grad_q_norm =  0.051507
grad_q_reg =  0.0058170
Optimizing q: remain iter - 829, q - 0.1999
grad_q_norm =  0.0023652
grad_q_reg =    4.7731e-05
Optimizing q: remain iter - 828, q - 0.1999
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 827, q - 0.1999
grad_q_norm = -0.25893
grad_q_reg =  0.038198
Optimizing q: remain iter - 826, q - 0.1999
grad_q_norm =  0.0029703
grad_q_reg =    1.7376e-04
Optimizing q: remain iter - 825, q - 0.1999
grad_q_norm =  0.32601
grad_q_reg =  0.020560
Optimizing q: remain iter - 824, q - 0.1999
grad_q_norm = -0.027629
grad_q_reg =  0.0017984
Optimizing q: remain iter - 823, q - 0.1999
grad_q_norm =  0.10668
grad_q_reg =  0.0078028
Optimizing q: remain iter - 822, q - 0.1999
grad_q_norm = -0.049994
grad_q_reg =  0.0073413
Optimizing q: remain iter - 821, q - 0.1999
gra

grad_q_reg =  0.0022105
Optimizing q: remain iter - 743, q - 0.1999
grad_q_norm =  0.44254
grad_q_reg =  0.024112
Optimizing q: remain iter - 742, q - 0.1999
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 741, q - 0.1999
grad_q_norm =  0.020281
grad_q_reg =  0.0059961
Optimizing q: remain iter - 740, q - 0.1999
grad_q_norm = -0.055755
grad_q_reg =  0.017703
Optimizing q: remain iter - 739, q - 0.1999
Optimizing q: remain iter - 739, q - 0.1999
grad_q_norm =  0.0081919
grad_q_reg =    7.9379e-04
Optimizing q: remain iter - 738, q - 0.1999
grad_q_norm =  0.10483
grad_q_reg =  0.0062766
Optimizing q: remain iter - 737, q - 0.1999
grad_q_norm =    1.5124e-16
grad_q_reg = 0
Optimizing q: remain iter - 736, q - 0.1999
grad_q_norm =  0.0011716
grad_q_reg =    5.5195e-05
Optimizing q: remain iter - 735, q - 0.1999
grad_q_norm = -0.059221
grad_q_reg =  0.019756
Optimizing q: remain iter - 734, q - 0.1999
grad_q_norm = -0.0028139
grad_q_reg =  0.0059941
Optimizing q: remain iter - 73

grad_q_norm =  0.025726
grad_q_reg =    3.1789e-04
Optimizing q: remain iter - 654, q - 0.1999
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 653, q - 0.1999
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 652, q - 0.1999
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 651, q - 0.1999
grad_q_norm = -0.0077043
grad_q_reg =  0.0012522
Optimizing q: remain iter - 650, q - 0.1999
grad_q_norm =  0.0097332
grad_q_reg =  0.010218
Optimizing q: remain iter - 649, q - 0.1999
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 648, q - 0.1999
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 647, q - 0.1999
grad_q_norm = -0.047749
grad_q_reg =  0.0025566
Optimizing q: remain iter - 646, q - 0.1999
Optimizing q: remain iter - 646, q - 0.1999
grad_q_norm =  0.0078343
grad_q_reg =  0.0045350
Optimizing q: remain iter - 645, q - 0.1999
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 644, q - 0.1999
grad_q_norm = -0.0067464

...


grad_q_reg = 0
Optimizing q: remain iter - 460, q - 0.1998
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 459, q - 0.1998
grad_q_norm = -0.082930
grad_q_reg =  0.010230
Optimizing q: remain iter - 458, q - 0.1998
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 457, q - 0.1998
grad_q_norm =  0.028596
grad_q_reg =  0.0050955
Optimizing q: remain iter - 456, q - 0.1998
grad_q_norm =   -8.7843e-17
grad_q_reg = 0
Optimizing q: remain iter - 455, q - 0.1998
grad_q_norm = -0.043518
grad_q_reg =    8.3893e-04
Optimizing q: remain iter - 454, q - 0.1998
grad_q_norm =  0.032337
grad_q_reg =  0.0043025
Optimizing q: remain iter - 453, q - 0.1998
Optimizing q: remain iter - 453, q - 0.1998
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 452, q - 0.1998
grad_q_norm =  0.0031115
grad_q_reg =  0.0017677
Optimizing q: remain iter - 451, q - 0.1998
grad_q_norm = -0.16810
grad_q_reg =  0.035210
Optimizing q: remain iter - 450, q - 0.1998
grad_q_norm =  0.47259
grad_q_r

Optimizing q: remain iter - 369, q - 0.1998
grad_q_norm = -0.11446
grad_q_reg =  0.030394
Optimizing q: remain iter - 368, q - 0.1998
grad_q_norm = -0.14033
grad_q_reg =  0.042041
Optimizing q: remain iter - 367, q - 0.1998
grad_q_norm = -0.14018
grad_q_reg =  0.033332
Optimizing q: remain iter - 366, q - 0.1998
grad_q_norm = -0.13751
grad_q_reg =  0.0075677
Optimizing q: remain iter - 365, q - 0.1998
grad_q_norm = -0.034511
grad_q_reg =  0.0017259
Optimizing q: remain iter - 364, q - 0.1998
grad_q_norm =  0.34922
grad_q_reg =  0.011976
Optimizing q: remain iter - 363, q - 0.1998
grad_q_norm =  0.066106
grad_q_reg =  0.0033239
Optimizing q: remain iter - 362, q - 0.1998
grad_q_norm =  0.023654
grad_q_reg =  0.0048615
Optimizing q: remain iter - 361, q - 0.1998
grad_q_norm = -0.085429
grad_q_reg =  0.037364
Optimizing q: remain iter - 360, q - 0.1998
grad_q_norm = -0.092494
grad_q_reg =  0.0082443
Optimizing q: remain iter - 359, q - 0.1998
grad_q_norm = -0.16424
grad_q_reg =  0.036856


grad_q_norm =  0.044081
grad_q_reg =  0.0052136
Optimizing q: remain iter - 279, q - 0.1998
grad_q_norm = -0.032176
grad_q_reg = -0.027453
Optimizing q: remain iter - 278, q - 0.1998
grad_q_norm = -0.069764
grad_q_reg =  0.014909
Optimizing q: remain iter - 277, q - 0.1998
Optimizing q: remain iter - 277, q - 0.1998
grad_q_norm =  0.014782
grad_q_reg =  0.0035337
Optimizing q: remain iter - 276, q - 0.1998
grad_q_norm = -0.058043
grad_q_reg =  0.0045269
Optimizing q: remain iter - 275, q - 0.1998
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 274, q - 0.1998
grad_q_norm = -0.31471
grad_q_reg =  0.017804
Optimizing q: remain iter - 273, q - 0.1998
grad_q_norm = -0.093835
grad_q_reg = -0.033096
Optimizing q: remain iter - 272, q - 0.1998
grad_q_norm =   -4.8708e-04
grad_q_reg =  0.0072299
Optimizing q: remain iter - 271, q - 0.1998
grad_q_norm =  0.0057749
grad_q_reg =  0.0071608
Optimizing q: remain iter - 270, q - 0.1998
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain i

Optimizing q: remain iter - 190, q - 0.1998
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 189, q - 0.1998
grad_q_norm = -0.053208
grad_q_reg =  0.0092615
Optimizing q: remain iter - 188, q - 0.1998
grad_q_norm =  0.066500
grad_q_reg =    9.9125e-04
Optimizing q: remain iter - 187, q - 0.1998
grad_q_norm =   -9.8049e-17
grad_q_reg = 0
Optimizing q: remain iter - 186, q - 0.1998
grad_q_norm =   -6.5134e-17
grad_q_reg = 0
Optimizing q: remain iter - 185, q - 0.1998
Optimizing q: remain iter - 185, q - 0.1998
grad_q_norm =   -4.2068e-17
grad_q_reg = 0
Optimizing q: remain iter - 184, q - 0.1998
grad_q_norm =  0.039409
grad_q_reg =  0.013163
Optimizing q: remain iter - 183, q - 0.1998
grad_q_norm =  0.15534
grad_q_reg =  0.023622
Optimizing q: remain iter - 182, q - 0.1998
grad_q_norm = -0.15005
grad_q_reg =  0.0083136
Optimizing q: remain iter - 181, q - 0.1998
grad_q_norm =  0.0021142
grad_q_reg =  0.0010301
Optimizing q: remain iter - 180, q - 0.1998
grad_q_norm = 0
grad_q_r

grad_q_reg =  0.019255
Optimizing q: remain iter - 101, q - 0.1997
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 100, q - 0.1997
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 99, q - 0.1997
grad_q_norm =  0.0084704
grad_q_reg =  0.0027782
Optimizing q: remain iter - 98, q - 0.1997
grad_q_norm = -0.045215
grad_q_reg =  0.021542
Optimizing q: remain iter - 97, q - 0.1997
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 96, q - 0.1997
grad_q_norm = -0.13559
grad_q_reg =  0.0030033
Optimizing q: remain iter - 95, q - 0.1997
grad_q_norm = -0.15849
grad_q_reg =  0.0061572
Optimizing q: remain iter - 94, q - 0.1997
grad_q_norm =  0.070445
grad_q_reg =  0.0078974
Optimizing q: remain iter - 93, q - 0.1997
Optimizing q: remain iter - 93, q - 0.1997
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 92, q - 0.1997
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 91, q - 0.1997
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 90, q -

In [31]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.2.4 Find multi q

In [32]:
criterion = 'TEGRBCM';

In [33]:
printf('Optimizing q\r\n');
iqs = q + 0.01 * rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=5000, lr=1e-8, lambda=1e6, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing qs: remain iter - 5000
qs =

   0.20826
   0.20883
   0.20530
   0.20140
   0.20904
   0.20101
   0.20157
   0.20572
   0.20139
   0.20018
   0.20632
   0.20171
   0.20510
   0.20649
   0.20525
   0.20511
   0.20165
   0.20319
   0.20750
   0.20383
   0.20263
   0.20277
   0.20297
   0.20875
   0.20740
   0.20394
   0.20584
   0.20725
   0.20328
   0.20527

qs_dist =    2.5540e-07
Optimizing qs: remain iter - 4999
qs =

   0.20826
   0.20883
   0.20530
   0.20140
   0.20904
   0.20101
   0.20157
   0.20572
   0.20139
   0.20018
   0.20632
   0.20171
   0.20510
   0.20649
   0.20525
   0.20511
   0.20165
   0.20319
   0.20750
   0.20383
   0.20263
   0.20277
   0.20297
   0.20875
   0.20740
   0.20394
   0.20584
   0.20725
   0.20328
   0.20527

qs_dist =    9.4052e-07
Optimizing qs: remain iter - 4998
qs =

   0.20826
   0.20883
   0.20530
   0.20140
   0.20904
   0.20101
   0.20157
   0.20572
   0.20139
   0.20018


   0.20277
   0.20297
   0.20875
   0.20740
   0.20394
   0.20584
   0.20725
   0.20328
   0.20527

qs_dist =    3.7804e-07
Optimizing qs: remain iter - 4980
qs =

   0.20826
   0.20883
   0.20530
   0.20140
   0.20904
   0.20101
   0.20157
   0.20572
   0.20139
   0.20018
   0.20632
   0.20171
   0.20510
   0.20649
   0.20525
   0.20511
   0.20165
   0.20319
   0.20750
   0.20383
   0.20263
   0.20277
   0.20297
   0.20875
   0.20740
   0.20394
   0.20584
   0.20725
   0.20328
   0.20527

qs_dist =    1.1624e-06
Optimizing qs: remain iter - 4979
qs =

   0.20826
   0.20883
   0.20530
   0.20140
   0.20904
   0.20101
   0.20157
   0.20572
   0.20139
   0.20018
   0.20632
   0.20171
   0.20510
   0.20649
   0.20525
   0.20511
   0.20165
   0.20319
   0.20750
   0.20383
   0.20263
   0.20277
   0.20297
   0.20875
   0.20740
   0.20394
   0.20584
   0.20725
   0.20328
   0.20527

qs_dist = 0
Optimizing qs: remain iter - 4978
qs =

   0.20826
   0.20883
   0.20530
   0.20140
   0.20904
   

   0.20319
   0.20750

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)





qs_dist = 0
Optimizing qs: remain iter - 4939
qs =

   0.20826
   0.20883
   0.20530
   0.20140
   0.20904
   0.20101
   0.20156
   0.20572
   0.20139
   0.20018
   0.20631
   0.20170
   0.20509
   0.20649
   0.20525
   0.20511
   0.20165
   0.20319
   0.20750
   0.20383
   0.20263
   0.20277
   0.20297
   0.20875
   0.20740
   0.20394
   0.20584
   0.20725
   0.20328
   0.20527

qs_dist =    7.0460e-08
Optimizing qs: remain iter - 4938
qs =

   0.20826
   0.20883
   0.20530
   0.20140
   0.20904
   0.20101
   0.20156
   0.20572
   0.20139
   0.20018
   0.20631
   0.20170
   0.20509
   0.20649
   0.20525
   0.20511
   0.20165
   0.20319
   0.20750
   0.20383
   0.20263
   0.20277
   0.20297
   0.20875
   0.20740
   0.20394
   0.20584
   0.20725
   0.20328
   0.20527

qs_dist =    2.4220e-08
Optimizing qs: remain iter - 4937
qs =

   0.20826
   0.20883
   0.20530
   0.20140
   0.20904
   0.20101
   0.20156
   0.20572
   0.20139
   0.20018
   0.20631
   0.20170
   0.20509
   0.20649
  

...


   0.20722
   0.20323
   0.20521

qs_dist =    7.4241e-07
Optimizing qs: remain iter - 461
qs =

   0.20826
   0.20883
   0.20528
   0.20131
   0.20903
   0.20098
   0.20151
   0.20565
   0.20137
   0.20011
   0.20620
   0.20159
   0.20505
   0.20636
   0.20522
   0.20510
   0.20157
   0.20306
   0.20732
   0.20379
   0.20255
   0.20275
   0.20289
   0.20870
   0.20727
   0.20386
   0.20579
   0.20722
   0.20323
   0.20521

qs_dist =    3.3039e-07
Optimizing qs: remain iter - 460
qs =

   0.20826
   0.20883
   0.20528
   0.20131
   0.20903
   0.20098
   0.20151
   0.20565
   0.20137
   0.20011
   0.20620
   0.20159
   0.20505
   0.20636
   0.20522
   0.20510
   0.20157
   0.20306
   0.20732
   0.20379
   0.20255
   0.20275
   0.20289
   0.20870
   0.20727
   0.20386
   0.20579
   0.20722
   0.20323
   0.20521

qs_dist = 0
Optimizing qs: remain iter - 459
qs =

   0.20826
   0.20883
   0.20528
   0.20131
   0.20903
   0.20098
   0.20151
   0.20565
   0.20137
   0.20011
   0.20620
   0.

qs =

   0.20826
   0.20883
   0.20528
   0.20131
   0.20903
   0.20098
   0.20151
   0.20565
   0.20137
   0.20011
   0.20620
   0.20159
   0.20505
   0.20636
   0.20522
   0.20510
   0.20157
   0.20306
   0.20732
   0.20379
   0.20254
   0.20274
   0.20288
   0.20870
   0.20727
   0.20386
   0.20579
   0.20722
   0.20322
   0.20521

qs_dist =    5.1584e-07
Optimizing qs: remain iter - 365
qs =

   0.20826
   0.20883
   0.20528
   0.20131
   0.20903
   0.20098
   0.20151
   0.20565
   0.20137
   0.20011
   0.20620
   0.20159
   0.20505
   0.20636
   0.20522
   0.20510
   0.20157
   0.20306
   0.20732
   0.20379
   0.20254
   0.20274
   0.20288
   0.20870
   0.20727
   0.20386
   0.20579
   0.20722
   0.20322
   0.20521

qs_dist =    1.3610e-07
Optimizing qs: remain iter - 364
qs =

   0.20826
   0.20883
   0.20528
   0.20131
   0.20903
   0.20098
   0.20151
   0.20565
   0.20137
   0.20011
   0.20620
   0.20159
   0.20505
   0.20636
   0.20522
   0.20510
   0.20157
   0.20306
   0.207

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




   0.20288
   0.20870
   0.20726
   0.20385
   0.20579
   0.20722
   0.20322
   0.20521

qs_dist =    7.7457e-08
Optimizing qs: remain iter - 153
qs =

   0.20826
   0.20883
   0.20528
   0.20131
   0.20902
   0.20098
   0.20151
   0.20564
   0.20137
   0.20010
   0.20619
   0.20158
   0.20505
   0.20635
   0.20521
   0.20510
   0.20156
   0.20305
   0.20732
   0.20379
   0.20254
   0.20274
   0.20288
   0.20870
   0.20726
   0.20385
   0.20579
   0.20722
   0.20322
   0.20521

Optimizing qs: remain iter - 153
qs =

   0.20826
   0.20883
   0.20528
   0.20131
   0.20902
   0.20098
   0.20151
   0.20564
   0.20137
   0.20010
   0.20619
   0.20158
   0.20505
   0.20635
   0.20521
   0.20510
   0.20156
   0.20305
   0.20732
   0.20379
   0.20254
   0.20274
   0.20288
   0.20870
   0.20726
   0.20385
   0.20579
   0.20722
   0.20322
   0.20521

Optimizing qs: remain iter - 153
qs =

   0.20826
   0.20883
   0.20528
   0.20131
   0.20902
   0.20098
   0.20151
   0.20564
   0.20137
   0.200

   0.20385
   0.20579
   0.20722
   0.20322
   0.20521

qs_dist =    1.9233e-07
Optimizing qs: remain iter - 58
qs =

   0.20826
   0.20883
   0.20528
   0.20131
   0.20903
   0.20098
   0.20151
   0.20564
   0.20137
   0.20010
   0.20619
   0.20158
   0.20505
   0.20635
   0.20521
   0.20510
   0.20156
   0.20305
   0.20731
   0.20379
   0.20254
   0.20274
   0.20288
   0.20870
   0.20726
   0.20385
   0.20579
   0.20722
   0.20322
   0.20521

qs_dist =    6.0160e-07
Optimizing qs: remain iter - 57
qs =

   0.20826
   0.20883
   0.20528
   0.20131
   0.20903
   0.20098
   0.20151
   0.20564
   0.20137
   0.20010
   0.20619
   0.20158
   0.20505
   0.20635
   0.20521
   0.20510
   0.20156
   0.20305
   0.20731
   0.20379
   0.20254
   0.20274
   0.20288
   0.20870
   0.20726
   0.20385
   0.20579
   0.20722
   0.20322
   0.20521

qs_dist =    6.8134e-08
Optimizing qs: remain iter - 56
qs =

   0.20826
   0.20883
   0.20528
   0.20131
   0.20903
   0.20098
   0.20151
   0.20564
   0.201

In [34]:
qs
save('./res/tegrbcmenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   0.20826
   0.20883
   0.20528
   0.20131
   0.20903
   0.20098
   0.20151
   0.20564
   0.20137
   0.20010
   0.20619
   0.20158
   0.20505
   0.20635
   0.20521
   0.20510
   0.20156
   0.20305
   0.20731
   0.20379
   0.20254
   0.20274
   0.20288
   0.20870
   0.20726
   0.20385
   0.20579
   0.20722
   0.20322
   0.20520

TEGRBCM: SMSE 0.00217910, MSLL -4.5234, NLPD -1.6765


## 1.3 GPoE vs TEGPoE

### 1.3.1 GPoE baseline

In [22]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GPoE
GPoE: SMSE 0.00101203, MSLL -0.3547, NLPD 2.4921


### 1.3.2 Grid q

In [23]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TEGPoE'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:2.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegpoe%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGPoE
TEGPoE: SMSE 0.00114609, MSLL 0.4856, NLPD 3.3325 - q   0.20
TEGPoE: SMSE 0.00112165, MSLL -0.9451, NLPD 1.9018 - q   0.40
TEGPoE: SMSE 0.00109086, MSLL -1.2563, NLPD 1.5905 - q   0.60
TEGPoE: SMSE 0.00105384, MSLL -1.0500, NLPD 1.7969 - q   0.80
TEGPoE: SMSE 0.00101203, MSLL -0.3547, NLPD 2.4921 - q   1.00
TEGPoE: SMSE 0.00096796, MSLL 1.0108, NLPD 3.8576 - q   1.20
TEGPoE: SMSE 0.00092452, MSLL 3.4517, NLPD 6.2986 - q   1.40
TEGPoE: SMSE 0.00088403, MSLL 7.7342, NLPD 10.5811 - q   1.60
TEGPoE: SMSE 0.00084784, MSLL 15.2737, NLPD 18.1206 - q   1.80
TEGPoE: SMSE 0.00081630, MSLL 28.6999, NLPD 31.5468 - q   2.00


### 1.3.3 Find single q

In [24]:
criterion = 'TEGPoE';

In [25]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=0.6);

Optimizing q
Msize =

 Columns 1 through 11:

    600    619    426   1016    528    358    580    958    195    718    703

 Columns 12 through 22:

    655    803    640    849    989    568    787    827    717    249    573

 Columns 23 through 30:

    263    480    710    488    453    442    584    222

Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 0.6000
grad_q_norm =  38.857
grad_q_reg = -48.252
Optimizing q: remain iter - 999, q - 0.6017
grad_q_norm =  43.286
grad_q_reg = -50.658
Optimizing q: remain iter - 998, q - 0.6034
grad_q_norm =  10.364
grad_q_reg = -23.760
Optimizing q: remain iter - 997, q - 0.6042
grad_q_norm =  44.380
grad_q_reg = -51.514
Optimizing q: remain iter - 996, q - 0.6060
grad_q_norm =  46.875
grad_q_reg = -66.324
Optimizing q: remain iter - 995, q - 0.6083
grad_q_norm =  10.215
grad_q_reg = -34.063
Optimizing q: remain iter - 994, q - 0.6095
grad_q_norm =  35.591
grad_q_reg = -47.850
Optimizing q: remain iter - 993

Optimizing q: remain iter - 910, q - 0.6892
grad_q_norm = -5.8639
grad_q_reg = -5.5100
Optimizing q: remain iter - 909, q - 0.6894
grad_q_norm = -0.13463
grad_q_reg = -9.0929
Optimizing q: remain iter - 908, q - 0.6897
grad_q_norm = -5.6044
grad_q_reg = -13.664
Optimizing q: remain iter - 907, q - 0.6902
grad_q_norm =  6.9246
grad_q_reg = -14.917
Optimizing q: remain iter - 906, q - 0.6907
grad_q_norm =  18.714
grad_q_reg = -27.310
Optimizing q: remain iter - 905, q - 0.6917
grad_q_norm =  6.3549
grad_q_reg = -14.640
Optimizing q: remain iter - 904, q - 0.6922
grad_q_norm =  13.006
grad_q_reg = -18.759
Optimizing q: remain iter - 903, q - 0.6928
grad_q_norm = -0.87400
grad_q_reg = -5.0950
Optimizing q: remain iter - 902, q - 0.6930
grad_q_norm = -47.682
grad_q_reg = -19.262
Optimizing q: remain iter - 901, q - 0.6936
grad_q_norm =  10.669
grad_q_reg = -18.265
Optimizing q: remain iter - 900, q - 0.6943
grad_q_norm =  2.8379
grad_q_reg = -14.116
Optimizing q: remain iter - 899, q - 0.69

Optimizing q: remain iter - 816, q - 0.7263
grad_q_norm = -5.0791
grad_q_reg = -11.456
Optimizing q: remain iter - 815, q - 0.7267
grad_q_norm = -6.4575
grad_q_reg = -13.647
Optimizing q: remain iter - 814, q - 0.7272
grad_q_norm = -4.8325
grad_q_reg = -0.89156
Optimizing q: remain iter - 813, q - 0.7272
grad_q_norm = -2.6145
grad_q_reg = -7.8938
Optimizing q: remain iter - 812, q - 0.7275
grad_q_norm =  1.6046
grad_q_reg = -12.347
Optimizing q: remain iter - 811, q - 0.7279
grad_q_norm =  3.5625
grad_q_reg = -6.5636
Optimizing q: remain iter - 810, q - 0.7281
grad_q_norm = -5.4534
grad_q_reg = -2.2595
Optimizing q: remain iter - 809, q - 0.7282
grad_q_norm = -3.8752
grad_q_reg = -16.194
Optimizing q: remain iter - 808, q - 0.7288
grad_q_norm = 10.0000
grad_q_reg = -17.350
Optimizing q: remain iter - 807, q - 0.7294
grad_q_norm = -15.983
grad_q_reg =  9.4704
Optimizing q: remain iter - 806, q - 0.7290
grad_q_norm = -2.7143
grad_q_reg =  9.6555
Optimizing q: remain iter - 805, q - 0.728

Optimizing q: remain iter - 722, q - 0.7345
grad_q_norm =  18.514
grad_q_reg = -35.234
Optimizing q: remain iter - 721, q - 0.7357
grad_q_norm = -29.930
grad_q_reg =  17.856
Optimizing q: remain iter - 720, q - 0.7351
grad_q_norm =  10.616
grad_q_reg = -17.961
Optimizing q: remain iter - 719, q - 0.7357
grad_q_norm = -3.4131
grad_q_reg =  4.9305
Optimizing q: remain iter - 718, q - 0.7355
grad_q_norm = -22.692
grad_q_reg =  6.2777
Optimizing q: remain iter - 717, q - 0.7353
grad_q_norm = -26.774
grad_q_reg =  24.544
Optimizing q: remain iter - 716, q - 0.7345
grad_q_norm = -13.497
grad_q_reg =  5.1976
Optimizing q: remain iter - 715, q - 0.7343
grad_q_norm = -1.5737
grad_q_reg = -6.9446
Optimizing q: remain iter - 714, q - 0.7345
grad_q_norm = -19.112
grad_q_reg =  9.7913
Optimizing q: remain iter - 713, q - 0.7342
grad_q_norm =  30.781
grad_q_reg = -32.846
Optimizing q: remain iter - 712, q - 0.7353
grad_q_norm = -17.680
grad_q_reg = -26.276
Optimizing q: remain iter - 711, q - 0.7362

Optimizing q: remain iter - 628, q - 0.7418
grad_q_norm = -14.535
grad_q_reg =  3.6754
Optimizing q: remain iter - 627, q - 0.7417
grad_q_norm = -6.5193
grad_q_reg = -8.5390


...


Optimizing q: remain iter - 457, q - 0.7485
grad_q_norm =  11.036
grad_q_reg = -14.714
Optimizing q: remain iter - 456, q - 0.7490
grad_q_norm = -17.272
grad_q_reg =  7.1281
Optimizing q: remain iter - 455, q - 0.7487
grad_q_norm = -4.9698
grad_q_reg = -0.72034
Optimizing q: remain iter - 454, q - 0.7488
grad_q_norm = -7.7000
grad_q_reg = -1.0146
Optimizing q: remain iter - 453, q - 0.7488
grad_q_norm = -13.405
grad_q_reg = -3.7571
Optimizing q: remain iter - 452, q - 0.7489
grad_q_norm =  10.342
grad_q_reg = -15.040
Optimizing q: remain iter - 451, q - 0.7494
grad_q_norm = -12.534
grad_q_reg =  1.5942
Optimizing q: remain iter - 450, q - 0.7494
grad_q_norm = -20.918
grad_q_reg =  10.589
Optimizing q: remain iter - 449, q - 0.7490
grad_q_norm =  19.182
grad_q_reg = -23.918
Optimizing q: remain iter - 448, q - 0.7498
grad_q_norm = -23.546
grad_q_reg = -19.366
Optimizing q: remain iter - 447, q - 0.7505
grad_q_norm = -1.4288
grad_q_reg = -6.3608
Optimizing q: remain iter - 446, q - 0.75

Optimizing q: remain iter - 363, q - 0.7540
grad_q_norm = -29.874
grad_q_reg =  22.193
Optimizing q: remain iter - 362, q - 0.7532
grad_q_norm = -20.983
grad_q_reg = -21.920
Optimizing q: remain iter - 361, q - 0.7540
grad_q_norm = -2.4490
grad_q_reg = -3.9848
Optimizing q: remain iter - 360, q - 0.7541
grad_q_norm =  14.753
grad_q_reg = -24.644
Optimizing q: remain iter - 359, q - 0.7550
grad_q_norm = -6.0912
grad_q_reg = -1.4165
Optimizing q: remain iter - 358, q - 0.7550
grad_q_norm = -34.578
grad_q_reg =  14.322
Optimizing q: remain iter - 357, q - 0.7545
grad_q_norm = -24.722
grad_q_reg =  13.464
Optimizing q: remain iter - 356, q - 0.7541
grad_q_norm = -14.530
grad_q_reg =  8.2199
Optimizing q: remain iter - 355, q - 0.7538
grad_q_norm = -23.037
grad_q_reg =  8.3787
Optimizing q: remain iter - 354, q - 0.7535
grad_q_norm =  6.1264
grad_q_reg = -9.6357
Optimizing q: remain iter - 353, q - 0.7538
grad_q_norm = -26.290
grad_q_reg =  15.082
Optimizing q: remain iter - 352, q - 0.7533

Optimizing q: remain iter - 269, q - 0.7545
grad_q_norm =  5.7566
grad_q_reg = -8.6037
Optimizing q: remain iter - 268, q - 0.7548
grad_q_norm = -20.009
grad_q_reg = -13.258
Optimizing q: remain iter - 267, q - 0.7553
grad_q_norm = -37.100
grad_q_reg =  8.7297
Optimizing q: remain iter - 266, q - 0.7550
grad_q_norm = -21.826
grad_q_reg =  13.790
Optimizing q: remain iter - 265, q - 0.7545
grad_q_norm = -40.054
grad_q_reg =  26.614
Optimizing q: remain iter - 264, q - 0.7536
grad_q_norm = -27.895
grad_q_reg =  5.5648
Optimizing q: remain iter - 263, q - 0.7534
grad_q_norm = -16.068
grad_q_reg =  7.6870
Optimizing q: remain iter - 262, q - 0.7531
grad_q_norm = -25.190
grad_q_reg = -8.2069
Optimizing q: remain iter - 261, q - 0.7534
grad_q_norm = -44.095
grad_q_reg = -13.459
Optimizing q: remain iter - 260, q - 0.7539
grad_q_norm =  2.0496
grad_q_reg =  0.99346
Optimizing q: remain iter - 259, q - 0.7538
grad_q_norm = -23.969
grad_q_reg =  18.080
Optimizing q: remain iter - 258, q - 0.753

Optimizing q: remain iter - 175, q - 0.7533
grad_q_norm = -14.442
grad_q_reg =  9.5587
Optimizing q: remain iter - 174, q - 0.7530
grad_q_norm = -37.937
grad_q_reg = -16.446
Optimizing q: remain iter - 173, q - 0.7535
grad_q_norm = -2.6254
grad_q_reg = -5.0766
Optimizing q: remain iter - 172, q - 0.7537
grad_q_norm = -8.5280
grad_q_reg = -1.6913
Optimizing q: remain iter - 171, q - 0.7538
grad_q_norm = -41.657
grad_q_reg =  30.512
Optimizing q: remain iter - 170, q - 0.7527
grad_q_norm = -18.783
grad_q_reg =  8.5385
Optimizing q: remain iter - 169, q - 0.7524
grad_q_norm = -11.139
grad_q_reg = -14.350
Optimizing q: remain iter - 168, q - 0.7529
grad_q_norm = -13.546
grad_q_reg =  20.187
Optimizing q: remain iter - 167, q - 0.7522
grad_q_norm = -26.348
grad_q_reg =  18.333
Optimizing q: remain iter - 166, q - 0.7516
grad_q_norm = -3.6583
grad_q_reg = -1.1533
Optimizing q: remain iter - 165, q - 0.7516
grad_q_norm = -7.9568
grad_q_reg = -5.4598
Optimizing q: remain iter - 164, q - 0.7518

Optimizing q: remain iter - 81, q - 0.7458
grad_q_norm = -6.7379
grad_q_reg = -10.282
Optimizing q: remain iter - 80, q - 0.7461
grad_q_norm = -20.871
grad_q_reg =  11.518
Optimizing q: remain iter - 79, q - 0.7457
grad_q_norm = -21.701
grad_q_reg =  1.8106
Optimizing q: remain iter - 78, q - 0.7457
grad_q_norm = -3.5418
grad_q_reg =  16.214
Optimizing q: remain iter - 77, q - 0.7451
grad_q_norm =  7.7054
grad_q_reg = -15.175
Optimizing q: remain iter - 76, q - 0.7456
grad_q_norm = -21.905
grad_q_reg =  10.540
Optimizing q: remain iter - 75, q - 0.7453
grad_q_norm =  5.1768
grad_q_reg = -15.391
Optimizing q: remain iter - 74, q - 0.7458
grad_q_norm = -32.817
grad_q_reg =  26.502
Optimizing q: remain iter - 73, q - 0.7449
grad_q_norm = -8.4894
grad_q_reg = -9.4147
Optimizing q: remain iter - 72, q - 0.7452
grad_q_norm =  1.7846
grad_q_reg = -10.117
Optimizing q: remain iter - 71, q - 0.7455
grad_q_norm = -10.309
grad_q_reg =  6.8014
Optimizing q: remain iter - 70, q - 0.7453
grad_q_norm

In [26]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.3.4 Find multi q

In [27]:
printf('Optimizing q\r\n');
iqs = q + 0.01* rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=5000, lr=1e-8, lambda=1e6, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing qs: remain iter - 5000
qs =

   0.74966
   0.75562
   0.74928
   0.75746
   0.74916
   0.74935
   0.75223
   0.75259
   0.75542
   0.75624
   0.75309
   0.75225
   0.75147
   0.75620
   0.75596
   0.75248
   0.75617
   0.74906
   0.75572
   0.75285
   0.75456
   0.75365
   0.75008
   0.74895
   0.75364
   0.75340
   0.74908
   0.75269
   0.75515
   0.75394

qs_dist =    5.1073e-04
Optimizing qs: remain iter - 4999
qs =

   0.74944
   0.75568
   0.74921
   0.75749
   0.74916
   0.74941
   0.75221
   0.75260
   0.75547
   0.75621
   0.75280
   0.75225
   0.75154
   0.75626
   0.75601
   0.75250
   0.75617
   0.74913
   0.75579
   0.75291
   0.75437
   0.75359
   0.75013
   0.74892
   0.75371
   0.75324
   0.74906
   0.75276
   0.75519
   0.75401

qs_dist =    4.8302e-04
Optimizing qs: remain iter - 4998
qs =

   0.74929
   0.75569
   0.74928
   0.75724
   0.74916
   0.74941
   0.75216
   0.75235
   0.75551
   0.75616


   0.74895
   0.75496
   0.75330
   0.75467
   0.75408
   0.75000
   0.74963
   0.75351
   0.75397
   0.75003
   0.75308
   0.75591
   0.75433

qs_dist =    5.7853e-04
Optimizing qs: remain iter - 4979
qs =

   0.74581
   0.75540
   0.75017
   0.75681
   0.74947
   0.74922
   0.75187
   0.75178
   0.75581
   0.75537
   0.75340
   0.75312
   0.75135
   0.75615
   0.75535
   0.75301
   0.75480
   0.74896
   0.75503
   0.75333
   0.75468
   0.75408
   0.75002
   0.74960
   0.75358
   0.75404
   0.75010
   0.75307
   0.75593
   0.75434

qs_dist =    6.3072e-04
Optimizing qs: remain iter - 4978
qs =

   0.74553
   0.75546
   0.75023
   0.75681
   0.74952
   0.74930
   0.75193
   0.75158
   0.75579
   0.75544
   0.75347
   0.75318
   0.75142
   0.75594
   0.75536
   0.75307
   0.75482
   0.74868
   0.75490
   0.75334
   0.75474
   0.75414
   0.75003
   0.74965
   0.75353
   0.75411
   0.75010
   0.75308
   0.75597
   0.75405

qs_dist =    7.5139e-05
Optimizing qs: remain iter - 4977
qs =

  

   0.75134
   0.75632
   0.75504
   0.75403
   0.75366
   0.75151
   0.75545
   0.75502
   0.75361
   0.75431
   0.74819

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




   0.75675
   0.75012
   0.74917
   0.75145
   0.75202
   0.75676
   0.75395
   0.75411
   0.75342
   0.75106
   0.75514
   0.75518
   0.75422
   0.75363
   0.74820
   0.75422
   0.75460
   0.75484
   0.75454
   0.75026
   0.75055
   0.75326
   0.75535
   0.75211
   0.75398
   0.75602
   0.75494

qs_dist =    6.4344e-04
Optimizing qs: remain iter - 4936
qs =

   0.73902
   0.75443
   0.75204
   0.75669
   0.75012
   0.74916
   0.75140
   0.75184
   0.75683
   0.75391
   0.75417
   0.75348
   0.75106
   0.75518
   0.75499
   0.75425
   0.75334
   0.74821
   0.75395
   0.75467
   0.75488
   0.75459
   0.74998
   0.75060
   0.75311
   0.75541
   0.75218
   0.75404
   0.75609
   0.75500

qs_dist =    5.3108e-04
Optimizing qs: remain iter - 4935
qs =

   0.73883
   0.75444
   0.75211
   0.75670
   0.75015
   0.74923
   0.75147
   0.75186
   0.75688
   0.75398
   0.75389
   0.75348
   0.75110
   0.75525
   0.75505
   0.75432
   0.75341
   0.74828
   0.75402
   0.75473
   0.75483
   0.75465


...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




   0.73354
   0.79811
   0.74811
   0.78922
   0.85776
   0.83552
   0.85750
   0.80083

qs_dist =    3.1372e-04
Optimizing qs: remain iter - 273
qs =

   0.58442
   0.74159
   0.88518
   0.71470
   0.81010
   0.74821
   0.72988
   0.71494
   0.83967
   0.69063
   0.78366
   0.76710
   0.73172
   0.76552
   0.72035
   0.84962
   0.68276
   0.75300
   0.72730
   0.84632
   0.74696
   0.79962
   0.73360
   0.79816
   0.74816
   0.78928
   0.85779
   0.83553
   0.85747
   0.80086

qs_dist =    3.6197e-04
Optimizing qs: remain iter - 272
qs =

   0.58442
   0.74164
   0.88522
   0.71465
   0.81012
   0.74829
   0.72996
   0.71491
   0.83950
   0.69072
   0.78373
   0.76716
   0.73179
   0.76547
   0.72039
   0.84968
   0.68282
   0.75302
   0.72736
   0.84632
   0.74703
   0.79968
   0.73368
   0.79821
   0.74822
   0.78935
   0.85784
   0.83555
   0.85751
   0.80074

qs_dist =    5.1660e-04
Optimizing qs: remain iter - 271
qs =

   0.58438
   0.74165
   0.88523
   0.71471
   0.81017
   0

   0.74836
   0.80142
   0.73352
   0.80001
   0.74858
   0.78971
   0.85870
   0.83730
   0.85964
   0.80203

qs_dist =    4.2936e-04
Optimizing qs: remain iter - 162
qs =

   0.58410
   0.74139
   0.88780
   0.71374
   0.81167
   0.74855
   0.73090
   0.71477
   0.84097
   0.69086
   0.78438
   0.76794
   0.73193
   0.76449
   0.72044
   0.85153
   0.68294
   0.75308
   0.72645
   0.84688
   0.74843
   0.80148
   0.73334
   0.80008
   0.74850
   0.78977
   0.85872
   0.83733
   0.85968
   0.80198

qs_dist =    3.3193e-04
Optimizing qs: remain iter - 161
qs =

   0.58406
   0.74143
   0.88784
   0.71380
   0.81167
   0.74858
   0.73087
   0.71483
   0.84100
   0.69072
   0.78443
   0.76800
   0.73201
   0.76454
   0.72052
   0.85153
   0.68299
   0.75314
   0.72630
   0.84690
   0.74850
   0.80154
   0.73327
   0.80014
   0.74850
   0.78983
   0.85877
   0.83738
   0.85972
   0.80200

qs_dist =    5.2138e-04
Optimizing qs: remain iter - 160
qs =

   0.58407
   0.74149
   0.88784
   0.

Optimizing qs: remain iter - 67
qs =

   0.58387
   0.74133


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [28]:
qs
save('./res/tegpoeenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   0.58428
   0.74202
   0.88981
   0.71255
   0.81383
   0.74879
   0.73051
   0.71427
   0.84123
   0.69127
   0.78560
   0.76920
   0.73133
   0.76311
   0.72066
   0.85395
   0.68336
   0.75280
   0.72694
   0.84633
   0.74921
   0.80206
   0.73340
   0.80158
   0.75019
   0.79053
   0.86059
   0.83994
   0.86207
   0.80090

TEGPoE: SMSE 0.00105162, MSLL -1.1761, NLPD 1.6707
